# TP4056 TEMP pin input
TP4056 datasheet says:

> Connecting TEMP pin to NTC thermistor’s output in Lithium ion battery pack. If TEMP pin’s voltage is below 45% or above 80% of supply voltage VIN for more than 0.15S, this means that battery’s temperature is too high or too low, charging is suspended.





In [3]:


vratio1 = 0.45
vratio2 = 0.80
ratio_tol1 = 0.01
ratio_tol2 = 0.05

# Battery University recommends charging only from 5°C to 45°C.
# Inputs for NTC MF58 (B=3950) (LCSC part C267088)
# NTC value at 45°C
Rntc_min = 4352

# NTC value at 5°C
Rntc_max = 25531

# Stepping controls

resistor_values = [
   100,
   120,
   150,
   200,
   220,
   270,
   300,
   330,
   390,
   470,
   510,
   560,
   680,
   820,
    1e3,
  1.2e3,
  1.5e3,
  1.8e3,
    2e3,
  2.2e3,
  2.4e3,
  2.7e3,
    3e3,
  3.3e3,
  3.6e3,
  3.9e3,
  4.7e3,
 4.99e3,
  5.1e3,
  5.6e3,
  6.2e3,
  6.8e3,
  7.5e3,
  8.2e3,
  9.1e3,
   10e3,
   12e3,
   13e3,
   15e3,
   18e3,
   20e3,
   22e3,
   24e3,
   27e3,
   30e3,
   33e3,
   39e3,
   47e3,
 49.9e3,
   51e3,
   56e3,
   68e3,
   75e3,
   82e3,
  100e3,
  120e3,
  150e3,
  200e3,
  220e3,
  300e3,
  330e3,
  470e3,
]

step = 100  # Step size
startR = 100  # Step start
maxR1R2 = 250e3  # Step end
stop = False

def get_ratio(Rntc, r1, r2):
    Rpara = (1.0 * Rntc * r2) / (Rntc + r2)
    Vratio = (1.0 * Rpara) / (Rpara + r1)
    return Vratio

def within_tol(num, val, tol):
    return val - tol <= num <= val + tol

def test_TP4056_temp(Rntc1, Rntc2, Ratio1, Ratio2, tol1, tol2):
    global found, stop
    op = []
    found = 0
    best_error = float('inf')
    best_msg = None
    for r1 in resistor_values:
        for r2 in resistor_values:
            VratioCalc1 = get_ratio(Rntc1, r1, r2)

            if within_tol(VratioCalc1, Ratio1, tol1):
                VratioCalc2 = get_ratio(Rntc2, r1, r2)

                if within_tol(VratioCalc2, Ratio2, tol2):
                    error = abs(VratioCalc1 - vratio1) + abs(VratioCalc2 - vratio2) 
                    msg = f"Found R1={r1} R2={r2} ratio1={VratioCalc1:.3f} ratio2={VratioCalc2:.3f}  error={error:.3f}"
                    print(msg)
                    op.append(f"{msg}\n")

                    if error < best_error:
                        best_error = error
                        best_msg = msg

                    found += 1

            if stop:
                break
        if stop:
            break

    with open("op.txt", "w") as fh:
        fh.write("".join(op))

    return best_msg

print(f"Looking for ratio1={vratio1} and ratio2={vratio2}")
best_msg=test_TP4056_temp(Rntc_min, Rntc_max, vratio1, vratio2, ratio_tol1, ratio_tol2)

print(f"\nFound: {found}\n\n")
print("R1 is the upper resistor and R2 is the lower resistor in the divider.")
print(f"Best: {best_msg}")


Looking for ratio1=0.45 and ratio2=0.8
Found R1=4700.0 R2=33000.0 ratio1=0.450 ratio2=0.754  error=0.046
Found R1=4700.0 R2=39000.0 ratio1=0.454 ratio2=0.767  error=0.038
Found R1=4700.0 R2=47000.0 ratio1=0.459 ratio2=0.779  error=0.030
Found R1=4700.0 R2=49900.0 ratio1=0.460 ratio2=0.782  error=0.028
Found R1=4990.0 R2=47000.0 ratio1=0.444 ratio2=0.768  error=0.038
Found R1=4990.0 R2=49900.0 ratio1=0.445 ratio2=0.772  error=0.033
Found R1=4990.0 R2=51000.0 ratio1=0.446 ratio2=0.773  error=0.031
Found R1=4990.0 R2=56000.0 ratio1=0.447 ratio2=0.778  error=0.024
Found R1=4990.0 R2=68000.0 ratio1=0.450 ratio2=0.788  error=0.012
Found R1=4990.0 R2=75000.0 ratio1=0.452 ratio2=0.792  error=0.009
Found R1=4990.0 R2=82000.0 ratio1=0.453 ratio2=0.796  error=0.007
Found R1=4990.0 R2=100000.0 ratio1=0.455 ratio2=0.803  error=0.008
Found R1=4990.0 R2=120000.0 ratio1=0.457 ratio2=0.808  error=0.015
Found R1=4990.0 R2=150000.0 ratio1=0.459 ratio2=0.814  error=0.023
Found R1=5100.0 R2=51000.0 ratio1=